In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data

from model import ResidualBlock, ResNet
from train_eval_util import train, evaluate, calculate_accuracy, epoch_time
from getCIFAR10 import train_data, valid_data, test_data

import time

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')  
else:
    device = torch.device('cpu')

print(f"Selected device: {device}")

Selected device: mps


In [3]:
BATCH_SIZE = 128

train_iterator = DataLoader(train_data, batch_size= BATCH_SIZE, shuffle=True)

valid_iterator =  DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

test_iterator =  DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [4]:
model = ResNet(ResidualBlock, [3, 3, 3]).to(device)


total_layers = sum([1 for _ in model.modules() 
    if isinstance(_, nn.Conv2d) or isinstance(_, nn.Linear)]) - 2 # subtract input and output layers
    
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total number of layers: {total_layers}")
print(f"Total number of parameters: {total_params}")

Total number of layers: 20
Total number of parameters: 272474


In [5]:

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)



In [6]:
num_epochs = 100
train_acc_history = []
train_loss_history = []
valid_acc_history = []
valid_loss_history = []

for epoch in range(num_epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
        
    end_time = time.time()

        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    train_acc_history.append(train_acc)
    train_loss_history.append( train_loss)
    valid_acc_history.append(valid_acc)
    valid_loss_history.append(valid_loss)

Epoch: 01 | Epoch Time: 0m 46s
	Train Loss: 1.553 | Train Acc: 41.98%
	 Val. Loss: 1.369 |  Val. Acc: 50.80%
Epoch: 02 | Epoch Time: 0m 46s
	Train Loss: 1.151 | Train Acc: 58.58%
	 Val. Loss: 1.070 |  Val. Acc: 61.70%
Epoch: 03 | Epoch Time: 0m 46s
	Train Loss: 1.000 | Train Acc: 64.17%
	 Val. Loss: 1.026 |  Val. Acc: 62.52%
Epoch: 04 | Epoch Time: 0m 46s
	Train Loss: 0.909 | Train Acc: 67.49%
	 Val. Loss: 1.027 |  Val. Acc: 64.61%
Epoch: 05 | Epoch Time: 0m 46s
	Train Loss: 0.849 | Train Acc: 69.77%
	 Val. Loss: 0.858 |  Val. Acc: 69.92%
Epoch: 06 | Epoch Time: 0m 46s
	Train Loss: 0.795 | Train Acc: 71.57%
	 Val. Loss: 1.459 |  Val. Acc: 59.79%
Epoch: 07 | Epoch Time: 0m 46s
	Train Loss: 0.749 | Train Acc: 73.62%
	 Val. Loss: 0.940 |  Val. Acc: 67.54%
Epoch: 08 | Epoch Time: 0m 46s
	Train Loss: 0.721 | Train Acc: 74.59%
	 Val. Loss: 0.803 |  Val. Acc: 73.28%
Epoch: 09 | Epoch Time: 0m 46s
	Train Loss: 0.688 | Train Acc: 75.87%
	 Val. Loss: 1.192 |  Val. Acc: 64.55%
Epoch: 10 | Epoch T